# Axiomatic Semantics

In **operational semantics**, e.g., big-step reduction, we define the semantics of the program as a transformation from a state to another. For example,  if we start the program with the state $\{ x\mapsto 3, y \mapsto 4 \}$, we get the new state $\{ x \mapsto 7, y \mapsto 2 \}$.
In contrast, in **axiomantic semantics**, the semantics of the program is a transformation from a _set_ of states to another set of states, i.e., from the states satisfying a precondition to the states satisfying the postcondition.
For example, if we start a (division) program with the precondition $y \ne 0$, at the termination of the program, we get the postcondition $r \equiv x/y$.



## Hoare Tripple

- By Tony Hoare
- Reasoning about program correctness using _pre_ and _post_ conditions
- The Hoare Tripple  {$P$} S {$Q$}:
   - $P$: a formula representing the _precondition_
   - $Q$: a formula representing the _postcondition_
   - S: a program (statement)
   - Read:  assume $P$ holds, if $S$ executes successfully (i.e., terminates), then $Q$ holds
- $P$ and $Q$: _specifications_ of the program $S$
    
- Partial Correctness: _assume_ S terminates
- Total Correctness: _require_ S terminates


## Examples of Hoare Tripples

### Valid tripples
- {True} x:=5 {$x \equiv 5$}
- {$x \equiv y$} x := x + 3 {$x \equiv y+3$}
- {x $>$ -1} x:=2*x + 3 {$x > 1$}
- {$x \equiv a$} if x $<$ 0 then x := -x {$x \equiv |a|$}
- {False} x:=3 {$x \equiv 8$} 


### In-class Questions:

1. {$x\equiv y$} ??? {$x \equiv y$}\
   A: skip

- {???} x:= y - 3 {$x \equiv 8$}\
   A: $y \equiv 11$

- {$x < 0$} while(x!=0) do x:=x - 1 {???} \
   A: any formula, because the loop terminates so anything would do.
   Note: this tripple is valid for partial correctness, but is _not_ valid for _total_ correctness.



## Strongest Postconditions

Consider the Hoare tripples below:
1. {$x \equiv 5$} x:=x*2 {$true$}
- {$x \equiv 5$} x:=x*2 {$x > 0$}
- {$x \equiv 5$} x:=x*2 {$x \equiv 10 \vee \equiv 5$}
- {$x \equiv 5$} x:=x*2 {$x \equiv 10$}

All of these tripples are valid, but the last one with the postcondition  $x \equiv$ 10 is the strongest, i.e., most precise.  In general, we want _strong postconditions_ for our specifications.  However, there could be cases when we do not need the strongest postconditions, e.g., if we only want to show that x remains positive, then the postcondition $x > 0$ would suffice.

## Definition
- In {P} S {Q}, Q is the strongest postcondition if $\forall Q'. \{P\} S \{Q'\} \wedge Q \Rightarrow Q'$. This says that Q is the strongest postcondition if (i) it is a postcondition in the tripple {P} S {Q} and it is stronger than any other postconditions Q' of the tripple {P} S {Q'}.

For the example above, x $\equiv$ 10 is the _strongest_ postcondition because
  - $x \equiv 10 \Rightarrow true$
  - $x \equiv 10 \Rightarrow x>0$
  - $x \equiv 10 \Rightarrow (x\equiv 10 \vee x\equiv 5)$
  - $x \equiv 10 \Rightarrow x\equiv 10$.
  - $x \equiv 10$ is also stronger than any other postconditions of this example (e.g., $x > 1,  x  \equiv 10 \vee x > 3, \dots$). 

Note that if **False** is a postcondition in the tripple {P} S {Q}, then it would be the strongest possible postcondition because False implies anything.




## Weakest Preconditions

Consider the Hoare tripples below:
1. {$x \equiv 5 \wedge y \equiv 10$} z := x/y {$z<1$}
- {$x<y \wedge y>0$} z := x/y {$z<1$}
- {$y\neq 0 \wedge x/y <1$ } z := x/y {$z<1$}
All of these triples are true, but the last one $y\neq 0 \wedge x/y <1$ is the most general or _weakest_ precondition.  In general, we want  _weak preconditions_ for our specifications because it allows us to run the programs with fewer assumptions or restrictions.

## Definition
- In {P} S {Q}, P is the weakest precondition if 
$\forall P'. \{P'\} S \{Q \} \wedge P' \Rightarrow P$.

  

## Program Verification

We can verify (partial) program correctness using Hoare Triples and weakest preconditions.
To prove {P} S {Q} is valid, i.e., to prove the program S is correct wrt to the precondition P and postcondition Q, we check $P \Rightarrow \text{wp}(S, Q)$, where the function _wp_ returns the weakest precondition allowing the program S to achieve the postcondition Q.
In the following, we define the wp's for different statements in the While language.


## WP's for While statements

|Statement Type | S  | $$\text{wp}(S, Q)$$ | Comment
|---------------|----|---------|---|
|Assignment     | x:= e |  $$Q_x^e$$ | replace all occurences of x in $Q$ with e |
|Skip           | skip | $$Q$$ | No effect |
|Sequential     | S1;S2 | $$\text{wp}(S1, \text{wp}(S2,Q))$$| |
|Conditional    | if b then S1 else S2 | $$b \Rightarrow \text{wp}(S1,Q)~\wedge~ \neg b \Rightarrow \text{wp}(S2,Q)$$| |
|Loop           | while b do S  | $$(I) \wedge (I \wedge B \Rightarrow \text{wp}(S,I)) ~\wedge~ (I \wedge \neg B \Rightarrow Q)$$ | $I$ is a user supplied _loop invariant_


### WP for Assignment Statement
The wp function $\text{wp}(x:=e, Q) = Q_x^e$ says that we can achieve wp of the assignment  x:= e wrt to the postcondition $Q$ by simply replacing all occurences of the variable $x$ in $Q$ with the expression $e$.

#### Examples
1. wp(x:=3, $x +y \equiv 10$) =  $(x +y \equiv 10)_x^3$ = $3 + y \equiv 10$ = $y = 7$
- wp(x:=3, $x +y > 0$) =  $(x +y > 0)_x^{3}$ = $y > -3$
- wp(x:= 3*y + z, $x * y - z > 0$)   =  $(x * y - z > 0)_x^{3*y+z}$ = $(3*y+z) * y - z > 0$

### WP for Sequential Statements
- wp(x := x + 3, $x \equiv z$) = $x + 3 \equiv z$
- wp(x := x + 1; y := y * x , $y \equiv 2 * z$) \
    = wp(x := x + 1 , $(y \equiv 2 * z)_y^{y*x}$) \
    = wp(x := x + 1 , $y*x \equiv 2 * z$) \
    = $(y*x \equiv 2 * z)_x^{x+1}$ \
    = $y*(x+1) \equiv 2 * z$
- wp(if x > 0 then y := x else y := 0,$y > 0$) = $x > 0 \Rightarrow x > 0 ~\wedge~ x \le 0 \Rightarrow 0 > 0$.  Note this wp would be False because in the else branch, after setting y := 0, it's not possible to get the post condition $y > 0$.


### WP for While Statement

Notice the weakest precondition for while statement
$$(I) ~\wedge~ (I \wedge B \Rightarrow \text{wp}(S,I)) ~\wedge~ (I \wedge \neg B \Rightarrow Q)$$
relies on a user-supplied loop invariant $I$.
Indeed, while we can automatically computes the wp's for other statements automatically, we need the user to provide us a loop invariant to compute the wp's for while loop. 


A loop invariant is a property that (i) holds upon the entrance of the loop and (ii) preserves after the execution of the loop body.
In essense, the loop invariant captures the semantics of the loop and thus allows us to replace the iteration details of the loop with the invariant.
Often, the loop invariant properties are expressed as logical formulas.

In the below example 
```c
i := 0;
while (i < N) do
  i := N;
```
$N \ge 0$ and $i \le N$ are valid loop invariants. Note, thus $N \ge 0 \wedge i \le N$ is also a loop invariant.  In contrast,  $i\equiv 0$ and $i \equiv N$ are not loop invariants.
In general, there are many possible loop invariants, e.g., $\text{True}, i \le N + 1, \dots$ in this example.  




## Sufficiently Strong Loop Invariants

Like postconditions, we generally want _strong_ loop invariants to represent the more precise semantics of the program. 
However, just like postconditions, determining the strongest (or even strong) loop invariants can be nontrivial, especially when we have to manually come up with them.
Fortunately, in many cass, we do not need to have strong loop invariants, but just _sufficiently_ strong to prove program specifications.
For example, if we want to prove a weak postcondition, then we only need sufficiently strong invariant to establish that proof.




## Example

We show that the _multiplication_ program below is correct with respect to the precondition $P = \{A \ge 0 \wedge B \ge 0 \}$ and $Q=\{z \equiv A*B\}$.
We will use the loop invariant $I = \{A*B = z + x*y\}$ to demonstrate this.

```C
// P = {A >= 0 && B >= 0}
x := A; 
y := B; 
z := 0; 
// L1
while x > 0 do
    if odd(x) then 
        z := z + y
    y := 2 ∗ y; 
    x := x / 2;   //integer division
    // L2

// L3: Q = {z == A*B}
```


1. Initial (at L1, when we about to enter the loop): 
    - $(A \ge 0 \wedge B \ge 0) \wedge (x \equiv A \wedge y \equiv B \wedge z \equiv 0) \Rightarrow (A*B \equiv x + y)$.  Checked!

- Terminate (at L3, when the loop exits):
    - $(A*B \equiv z + x*y \wedge x \equiv 0) \Rightarrow (A*B \equiv z)$. Checked!
    - Note, the loop exit is $x \le 0$, however we know x cannot be negative so we strengthen it to $x 
\equiv 0$

- Loop (at L2, the loop inv is preseved at the end of the loop body): we show that $(I \wedge x > 0) \Rightarrow \text{wp}(\text{loop_body}, I)$.  In this example, the loop body consists of 2 cases:
   - Case 1: when x is even:
       - $x > 0 \wedge A*B = z + x *y \Rightarrow ((A*B = z + x*y)_y^{2*y})_x^{x/2}$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow (A*B = z + (x/2)*2*y)$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow A*B = z + x*y$.  Checked

   - Case 2: when x is odd, then due to intger division, we have $x = 2*u + 1$, e.g., if $x = 7$, then $u = 3$:
       - $x > 0 \wedge A*B = z + x *y \Rightarrow (((A*B = z + x*y)_y^{2*y})_x^{x/2})_z^{z+y}$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow (A*B = (z+y) + (x/2)*2*y)$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow (A*B = (z+y) + ((2u + 1)/2)*2*y)$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow (A*B = (z+y) + (2u/2 + 1/2)*2*y)$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow (A*B = z + y + u * 2*y)$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow (A*B = z + y * (1 + u * 2))$ \
       = $x > 0 \wedge A*B = z + x *y \Rightarrow (A*B = z + y * x)$.  Checked
       
We also note that this program terminates because (i) if $x \le 0$, we skip the loop and (ii) otherwise, ie.., $x > 0$, we enter the loop but at each iteration we have $x / 2$ and thus eventually this integer division will result in $x = 0$ and we exit the loop.



## Example

Given the below program with the precondition $P = \{x \le 9\}$ and the postcondition $Q = \{x \equiv 12\}$
```C
    x := x + 1;
    while x != 10 do
       x := x + 1;
    x := x + 2
```

1. Find an invariant $I$ for the loop (that hopefully is sufficiently strong to prove $Q$)
   - Guess $x \le 10$

- Find the *wp* of the loop
   - wp(x := x + 1; while x != 10 do x := x + 1; x := x + 2, $x \equiv 12$) \
    = wp(x := x + 1; while x != 10 do x := x + 1, $x \equiv 10$) \
    = wp(x := x + 1, wp(while x != 10 do x := x + 1, $x \equiv 10$) \
    = wp(x := x + 1, $(x\le 10) ~\wedge~ (x \le 10 \wedge x \ne 10 \Rightarrow \text{wp}(\text{x := x + 1}, x \le 10)) ~\wedge~ (x \le 10 \wedge x \equiv 10 \Rightarrow x \equiv 10)$) \
    = wp(x := x + 1, $x\le 10 ~\wedge~ \text{True} ~\wedge~ \text{True}$
    = $x \le 9$

- Prove the program is correct
   - $P \Rightarrow$  wp(x:= x + 1; while x != 10 do x := x + 1; x := x + 2, $x \equiv 10$) \
   = $x \le 9 \Rightarrow x\le 9$ \
   = True
- Argue the program terminates
  - The postcondition states that $x \le 10$.  If $x \equiv 10$, then we skips the loop and terminates the program.  Otherwise ($x < 10$), $x$ is incremented by 1 at each iteration and thus eventually we will get $x == 10$, exit the loop, and terminate.



## Example

Given the below program with the precondition $P=\{ N \le 10 \}$
```c
i := 0;
while (i < N) do
  i := N
```

1. Determine what this program does and write a post condition $Q$ for it
  - $Q = \{ i \equiv N \}$
  
- Find an invariant $I$ for the loop that is sufficiently strong to prove $Q$
  - $i \le N$.  Note: something like $N \le 0$ is not sufficient
  
- Prove that this program is correct with respect to $P$ and $Q$
    1. First, we find the wp of the program (we would use $I$ for the loop) \ 
     wp(i := 0; while i < 10 do i := N, $i \le N$) \
       = wp(i := 0, wp(while i < 10 do i := N, $i \le N$))\
       = wp(i := 0, $(i \le N \wedge i < 10 \Rightarrow i \le N) ~\wedge~ (i \le N \wedge i \ge N \Rightarrow i \equiv N)$)\
       = $(0 \le N \wedge 0 < 10 \Rightarrow 0 \le N) ~\wedge~ (0 \le N \wedge 0 \ge N \Rightarrow 0 \equiv N)$

    - Then, we show that $P \Rightarrow \text{wp}$\
    $N \le 10 \Rightarrow (0 \le N \wedge 0 < 10 \Rightarrow 0 \le N) ~\wedge~ (0 \le N \wedge 0 \ge N \Rightarrow 0 \equiv N) = \text{True}$ \
    - In fact, note that the wp of the loop can be simplified to just True and thus it does not matter what the precondition or the assignment statement before the loop is.

- Argue that this program terminates